In [2]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/markinghack/products.csv
/kaggle/input/markinghack/retail_outlets.csv
/kaggle/input/markinghack/turnover_in.csv
/kaggle/input/markinghack/turnover_out.csv
/kaggle/input/markinghack/transaction_data.csv
/kaggle/input/markinghack/participants.csv


In [3]:
# df_in = pd.read_csv("/kaggle/input/markinghack/turnover_in.csv")
df_out = pd.read_csv("/kaggle/input/markinghack/turnover_out.csv")
df_participations = pd.read_csv('/kaggle/input/markinghack/participants.csv')
df_participations

,inn,region_code
0,60DFB6F87ABF6232A5D97A1A827F8D88,1
1,2D13C279C7CEC509D21695F0268ECCEA,1
2,1705C73792D7A1A5ABDF1883FE851004,1
3,B4CC35BD7D8F69E665160F548EBE125D,23
4,2DB318BEA4A44280FA488A0B2AF36254,1
...,...,...
23467,FD378AE7B1D03A5B949A96BF2715B941,77
23468,CB8661DA918DFA68498D7F929C426E1E,99
23469,92ECDE80D71E459551E809AF332785AA,77
23470,86B3B09285C227598C66DE143C5AD336,78


In [4]:
df_out

,dt,gtin,prid,inn,id_sp_,type_operation,price,cnt
0,2021-11-22,5A3E5F7B2D093D1D6CB3CF93BA9AC8A6,DC7432556B58D454B4E37F0F0D1E663D,B035ED5D308BC70E5EA9342B6DBF5810,19485F700068E6A9A8593AFDE9803638,Продажа конечному потребителю в точке продаж,3500,1
1,2021-11-22,9A2202A48FC5E462C44B78B0C79E1DBD,D18B1B470E57871FAB5B4E949E256BB4,DD486FED86564603D2CD7CF9EED2C458,C2969A4C8A40C12902EAD4936E303045,Продажа конечному потребителю в точке продаж,56714,1
2,2021-11-22,877ABE9DE14E0B510DFC8DE63EEFA817,62B5EAEF59E6693F44B1074C1140BF45,62B5EAEF59E6693F44B1074C1140BF45,5637334F301773E28AE8730713E6D5F7,Продажа конечному потребителю в точке продаж,74314,1
3,2021-11-22,B004F889F17F8AB365C494B3D0D06E1A,D7326D87A37D73AF19AF53668A7116D6,D7326D87A37D73AF19AF53668A7116D6,0FF56DEC16050D3B6372F2CB1BD6DFCB,Продажа конечному потребителю в точке продаж,4999,1
4,2021-11-22,E1C0503EA9201D56D7173CB4CA4A6AFA,50955BB0B97298EEAEB1F6CF9712F37A,39FFA592BE77CA7DE37C65C1B8423E8C,9E660AA1DE428BA88D1942DB2824BDAB,Продажа конечному потребителю в точке продаж,29857,1
...,...,...,...,...,...,...,...,...
52095176,2022-04-07,5D74B67E1DAC42C7526F27A6BDA73FF6,DA62EC79660CF21AC37A260DA6F642C4,E147E089C99F3A5EFEEF57739F60EFD1,09C6904B8ACA75E3380000EB4AC0A616,Продажа конечному потребителю в точке продаж,1128,2
52095177,2022-04-07,B1C38C41D23BA4AC631FCF073744A362,D80CBF60BE36ACE344F8350B648C688D,E147E089C99F3A5EFEEF57739F60EFD1,NaN,Продажа конечному потребителю в точке продаж,3571,1
52095178,2022-04-07,2C654E2727AACB613F30DB2FF30844A1,6BC47888CB4E777EE8DF0B940FFA18C7,E147E089C99F3A5EFEEF57739F60EFD1,2FF1ACF094C477DF28C6059CA11424BF,Продажа конечному потребителю в точке продаж,4196,1
52095179,2022-04-07,859C541431985132C29F457945761242,DA62EC79660CF21AC37A260DA6F642C4,E147E089C99F3A5EFEEF57739F60EFD1,07905E35AB800714D847D6DEBAE46F47,Продажа конечному потребителю в точке продаж,985,3


In [20]:
def calc_region_price_product(df_out, df_participations, gtin=None, region_code=None):
    """Calculation of the market price for this product by region
    
    Args:
    * df_out: df: data on the withdrawal of goods from circulation.
    * df_participations: df: directory of product turnover participants.
    * gtin: list or tuple or series of strings: if None return all.
    * region_code: list or tuple or series of floats: if None return all.
    
    Return:
    * DataFrame with columns: gtin, region_code, price (median, mean, min, max, count).
    """
    if region_code is not None:
        df_participations = df_participations[df_participations.region_code.isin(region_code)]
    if gtin is not None:
        df = df_out.loc[df_out.gtin.isin(gtin), ("gtin", "price", "inn")]
    else:
        df = df_out[["gtin", "price", "inn"]]
    df = df.merge(df_participations, how="left")\
        [df_out.type_operation != "Списание / Вывод из оборота без получателя"]
    df_price_reg = df[["gtin", "price", "region_code"]]
    return df_price_reg.groupby(["gtin", "region_code"]).agg({
        "price": ["median", "mean", "min", "max", "count"]
    })

CPU times: user 39.6 s, sys: 9.71 s, total: 49.3 s
Wall time: 49.2 s


In [44]:
calc_region_price_product(df_out, df_participations, ["000083D6DD3B1F4374EBEF0C7C50BDE6", "FFFFACED9831D8E1A9F5F9C4FDFCDFEB"])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == "__main__":


price                         \
                                                median           mean     min   
gtin                             region_code                                    
000083D6DD3B1F4374EBEF0C7C50BDE6 77           151428.0  140668.666667  111150   
FFFFACED9831D8E1A9F5F9C4FDFCDFEB 2              5714.0    5714.000000    5714   
                                 3             13571.0   11416.333333       0   
                                 5             17142.0   17142.000000   17142   
                                 13             9714.0    8249.666667    5321   
                                 14             6785.0    6785.000000    6785   
                                 16             4457.0    4805.250000    4400   
                                 21             6942.5    6873.333333    5571   
                                 22             8509.0    7435.599246       0   
                                 23             3540.0    8226.870968    3471   
                                 24             5444.5    4265.416667      71   
                                 25             6414.0    6417.500000    6414   
                                 26             4636.0    6094.600000    4485   
                                 29             6606.5    6856.750000    6500   
                                 31             7042.0    7182.833333    5000   
                                 32            10271.0   10271.000000   10271   
                                 33             6414.0    6414.000000    6414   
                                 34             6107.0    6124.750000    5357   
                                 35             6285.0    6181.266667    4142   
                                 36             9142.0    9142.000000    9142   
                                 37             5928.0    6400.285714    4269   
                                 38             7500.0    7561.810811       0   
                                 39             9500.0    9500.000000    9500   
                                 40             7357.0    7817.071429    5885   
                                 42             6000.0    6142.666667    5857   
                                 43             6414.0    6367.500000    6071   
                                 46             3214.0    3821.125000       0   
                                 47             7892.5    8053.000000    7142   
                                 48             6842.0    7060.000000    6842   
                                 50                0.0    2073.038462       0   
                                 52             6714.0    6234.571429       0   
                                 53             6999.5    6999.500000    6857   
                                 56             7667.0    7406.250000    6057   
                                 57             6500.0    6500.000000    6500   
                                 59             6107.0    5271.777778       0   
                                 61             5314.0    5524.071429    4642   
                                 62             5942.0    6356.333333    5942   
                                 63             6414.0    6131.750000    5285   
                                 65                0.0       0.000000       0   
                                 66            10000.0    9664.052632    4757   
                                 67             5714.0    5820.800000    4106   
                                 68            10635.0   10631.750000    8514   
                                 71             8071.0    8539.761905    5571   
                                 75             8200.0    8039.333333    5171   
                                 76             6371.0    6575.666667    5571   
                                 77             7357.0    9139.131313       0   
                                 78             5142.0    4448.795918       0   

In [45]:
calc_region_price_product(df_out, df_participations, ["000083D6DD3B1F4374EBEF0C7C50BDE6", "FFFFACED9831D8E1A9F5F9C4FDFCDFEB"], [77])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == "__main__":


price                         \
                                                median           mean     min   
gtin                             region_code                                    
000083D6DD3B1F4374EBEF0C7C50BDE6 77.0         151428.0  140668.666667  111150   
FFFFACED9831D8E1A9F5F9C4FDFCDFEB 77.0           7357.0    9139.131313       0   

                                                            
                                                 max count  
gtin                             region_code                
000083D6DD3B1F4374EBEF0C7C50BDE6 77.0         159428     3  
FFFFACED9831D8E1A9F5F9C4FDFCDFEB 77.0          19985   396